In [1]:
# Import dependencies
import subprocess
import json
import os
from dotenv import load_dotenv
from pprint import pprint


# Import constants.py and necessary functions from bit (for BTCTest) and web3 (for ETH)
from constants import *
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

from web3 import Web3, middleware, Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware
from eth_account import Account

# connect Web3 to our local network 8545
w3 = Web3(Web3.HTTPProvider('http://localhost:8545')) 


# enable PoA middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

# set gas price strategy to built-in "medium" algorythm (default to about 5 per tx)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

# Load and set environment variables
load_dotenv('./ft_mnemonic.env')

# Set your mnemonic variable
mnemonic=os.getenv("ft_mnemonic")

In [2]:
# Create a function called `derive_wallets`
def derive_wallets(mnemonic, coin, depth):
    command = f'./derive -g --mnemonic="{mnemonic}" --coin="{coin}" --numderive="{depth}" --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [3]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    "ETH": derive_wallets(mnemonic, ETH, "3"),
    "BTC-TEST": derive_wallets(mnemonic, BTCTEST, "3")
}

In [4]:
print(json.dumps(coins, indent=4))

{
    "ETH": [
        {
            "path": "m/44'/60'/0'/0/0",
            "address": "0xf46e2491686bF2E15918Fc316dee66f600446f26",
            "xprv": "xprvA2UpcB9biNZnDYVtZPF8wHMzNrPT8BPTinkdwhNGJhJ1UdPsMy848Tbd4zben5aqZneHhrtUEmSx9kQfoFRMZ4wmXjUHYRVkF9dMwCLdRrJ",
            "xpub": "xpub6FUB1ggVYk85S2aMfQn9JRJivtDwXe7K61gEk5mss2pzMRj1uWSJgFv6vH6t1UADUr18BmrotSo8LMiLfoVSHg9fPL7JDuSqPRNwej47pqd",
            "privkey": "0x82db79317f8f4e22a27e581c4ed56c4da09851375c92619672b0fee289f4d477",
            "pubkey": "0344e8b1bc90307872f144250eaef76381a84a4b11cc5e517e5034c0135a648e78",
            "pubkeyhash": "361447c0a97479dd755bf902b8a71b67fe6fda0a",
            "index": 0
        },
        {
            "path": "m/44'/60'/0'/0/1",
            "address": "0xD8475F9206fe8Cfb5481aa7D28A41e7A29084F68",
            "xprv": "xprvA2UpcB9biNZnGfHGUnKFKFb9B5VB6XmDcqdt49LULdQwQTFAgtvMcyzLrPPWa2p5GSwqHFihf1N3qbvXexwuYrTXmwKmxQtpvboq7V1FqcU",
            "xpub": "xpub6FUB1ggVYk85V9MjaorFgPXsj7Kf

In [5]:
# Extract the two private keys from the two first addresses we will be using to initiate our test transactions
btc_privkey=coins['BTC-TEST'][0]['privkey']
eth_privkey= coins['ETH'][0]['privkey']

In [6]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [7]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    if coin == ETH:
        value = w3.toWei(amount, 'ether') # convert eth to wei
        gasEstimate = w3.eth.estimateGas({'to':to, 'from':account, 'amount':value})
        return{
            'to': to,
            'from': account,
            'value': value,
            'gas': gasEstimate,
            'gasPrice': w3.eth.generateGasPrice(),
            'nonce': w3.eth.getTransactionCount(account),
            'chainId': w3.eth.chain_id
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [9]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account.address, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

In [10]:
# Create the priv_key object that we will need to send transactions
account_btc1 = priv_key_to_account(BTCTEST, btc_privkey)
account_eth1 = priv_key_to_account(ETH, eth_privkey)

## BTCTEST test transaction

In [11]:
# Check that we are correctly connected to our local blockchain. 
w3.isConnected()

True

In [13]:
# Generate and send the BTC transaction
send_tx(BTCTEST,account_btc1 ,'n181xfYwhZ6W6WHXdAGzYinZqSiNR7DK7v', 0.001)

## ETH test transaction

In [14]:
# Check the Balance of the first address on our local network - this will be the address we use to send the transaction
# Note since this address was new to our local Network, we had to first found it as the balance was at 0.
# To do so you could either send funds via MyCrypto from another address or the nodes directly - OR reinitialize a new network and prefunding this address
w3.eth.getBalance("0xf46e2491686bF2E15918Fc316dee66f600446f26")

99849999979000000000000

In [15]:
# Generate and send the ETH transaction
send_tx(ETH, account_eth1, '0xD8475F9206fe8Cfb5481aa7D28A41e7A29084F68', 150)

HexBytes('0xf340f4ffa2cd566f382dc39c67307bf1ace8d32816775de7115433e123bfc4f2')

In [17]:
# confirm that the recipient actually received the 150 ETH by checking its balance
w3.eth.getBalance("0xD8475F9206fe8Cfb5481aa7D28A41e7A29084F68")

300000000000000000000